In [1]:
import torch
import numpy
import PIL.Image
from tha3.util import resize_PIL_image,extract_PIL_image_from_filelike, extract_pytorch_image_from_PIL_image
from tqdm import tqdm
import onnx
from onnxsim import simplify
import onnxruntime as ort
import numpy as np
from torch import Tensor
from torch.nn import Module
from typing import List, Optional
from torch.nn.functional import interpolate
import onnx_tool

MODEL_NAME = "separable_half"
HALF = True
DEVICE_NAME = 'cuda:0'
IMAGE_INPUT = "data\images\crypko_07.png"
USE_RANDOM_IMAGE = False

providers = [("CUDAExecutionProvider", {"device_id": 0, #torch.cuda.current_device(),
                                        "user_compute_stream": str(torch.cuda.current_stream().cuda_stream)})]
sess_options = ort.SessionOptions()

device = torch.device(DEVICE_NAME)
dtype = torch.float16 if HALF else torch.float32


In [2]:
#Prepare models
def load_poser(model: str, device: torch.device):
    print("Using the %s model." % model)
    if model == "standard_float":
        from tha3.poser.modes.standard_float import create_poser
        return create_poser(device)
    elif model == "standard_half":
        from tha3.poser.modes.standard_half import create_poser
        return create_poser(device)
    elif model == "separable_float":
        from tha3.poser.modes.separable_float import create_poser
        return create_poser(device)
    elif model == "separable_half":
        from tha3.poser.modes.separable_half import create_poser
        return create_poser(device)
    else:
        raise RuntimeError("Invalid model: '%s'" % model)
        
poser = load_poser(MODEL_NAME, DEVICE_NAME)
pose_size = poser.get_num_parameters()

eyebrow_decomposer = poser.get_modules()['eyebrow_decomposer']
eyebrow_morphing_combiner = poser.get_modules()['eyebrow_morphing_combiner']
face_morpher = poser.get_modules()['face_morpher']
two_algo_face_body_rotator = poser.get_modules()['two_algo_face_body_rotator']
editor = poser.get_modules()['editor']

Using the separable_half model.
Loading the eyebrow decomposer ... DONE!!!
Loading the eyebrow morphing conbiner ... 

F:\talking-head-anime-3-demo\tha3\util.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f)


DONE!!!
Loading the face morpher ... DONE!!!
Loading the face-body rotator ... DONE!!!
Loading the combiner ... DONE!!!


In [3]:
#Prepare one pass inference image data
pt_img = None
if USE_RANDOM_IMAGE:
    pt_img = torch.rand(1, 4, 512, 512,dtype=dtype, device=device) * 2.0 - 1.0
else:
    pil_image = resize_PIL_image(extract_PIL_image_from_filelike(IMAGE_INPUT), size=(512,512))
    
    if HALF:
        pt_img = extract_pytorch_image_from_PIL_image(pil_image).half().reshape(1,4,512,512).to(DEVICE_NAME)
    else:
        pt_img = extract_pytorch_image_from_PIL_image(pil_image).reshape(1,4,512,512).to(DEVICE_NAME)
zero_pose = torch.zeros(1, pose_size, dtype=dtype, device=device)

poser_torch_res = poser.pose(pt_img, zero_pose)

In [4]:
#Small bench for whole pose
from time import time
t1 = time()
for i in tqdm(range(100)):
    poser.pose(pt_img, zero_pose)
print(time() - t1)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.84it/s]

3.471764326095581


In [5]:
# Work on eyebrow decomposer export and quantize
EYEBROW_DECOMPOSER_ONNX_MODEL_NAME = "eyebrow_decomposer.onnx"
EYEBROW_DECOMPOSER_SIMPLIFIED_ONNX_MODEL_NAME = "eyebrow_decomposer_sim.onnx"
EYEBROW_DECOMPOSER_INPUT_LIST = ['input_image']
EYEBROW_DECOMPOSER_OUTPUT_LIST = ["background_layer", "eyebrow_layer"]
EYEBROW_DECOMPOSER_INPUT_SHAPE = (1,4,128,128)
EYEBROW_DECOMPOSER_INPUT_IMAGE_SIZE = 128

eyebrow_decomposer_input_img = pt_img[:,:, 64:192, 64 + 128:192 + 128].reshape(EYEBROW_DECOMPOSER_INPUT_SHAPE)
eyebrow_decomposer_input_img_numpy = eyebrow_decomposer_input_img.cpu().numpy()
eyebrow_decomposer_torch_res = eyebrow_decomposer(eyebrow_decomposer_input_img) # Try one round inference to catch problem

In [6]:
class EyebrowDecomposerWrapper(Module):
    def __init__(self, eyebrow_decomposer_obj):
        super().__init__()
        self.eyebrow_decomposer = eyebrow_decomposer_obj
    def forward(self, image: Tensor, *args) -> List[Tensor]:
        cropped = image[:,:, 64:192, 64 + 128:192 + 128].reshape((1,4,128,128))
        decomposer_res = self.eyebrow_decomposer(cropped)
        return [decomposer_res[3], decomposer_res[0]]
eyebrow_decomposer_wrapper = EyebrowDecomposerWrapper(eyebrow_decomposer).eval()
eyebrow_decomposer_wrapped_torch_res = eyebrow_decomposer_wrapper(pt_img)

In [7]:
#Export onnx model finally get a simplified decomposer onnx model
torch.onnx.export(eyebrow_decomposer_wrapper,               # model being run
                  pt_img,                         # model input (or a tuple for multiple inputs)
                  EYEBROW_DECOMPOSER_ONNX_MODEL_NAME,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = EYEBROW_DECOMPOSER_INPUT_LIST,   # the model's input names
                  output_names = EYEBROW_DECOMPOSER_OUTPUT_LIST) # the model's output names
onnx_model = onnx.load(EYEBROW_DECOMPOSER_ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
onnx_model_sim, check = simplify(onnx_model)
if check:
    onnx.save(onnx_model_sim, EYEBROW_DECOMPOSER_SIMPLIFIED_ONNX_MODEL_NAME)
else:
    print("Simplify error!")

F:\Anaconda\envs\talking-head-anime-3-dev\lib\site-packages\torch\onnx\symbolic_helper.py:1531: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(


In [8]:
input_dict = {k:v.cpu().detach().numpy() for k,v in zip(EYEBROW_DECOMPOSER_INPUT_LIST, (pt_img))}
# Verify correctness compare to pytorch
pt_img_np = pt_img.cpu().detach().numpy()
ort_sess = ort.InferenceSession(EYEBROW_DECOMPOSER_SIMPLIFIED_ONNX_MODEL_NAME, sess_options=sess_options, providers=providers)
onnx_sim_output = ort_sess.run(None, {'input_image':pt_img_np,})
print("MSE is: ",((onnx_sim_output[0] - eyebrow_decomposer_torch_res[3].cpu().detach().numpy()) ** 2).mean())
print("MSE is: ",((onnx_sim_output[1] - eyebrow_decomposer_torch_res[0].cpu().detach().numpy()) ** 2).mean())

MSE is:  0.0
MSE is:  0.0


In [9]:
onnx_tool.model_profile(EYEBROW_DECOMPOSER_SIMPLIFIED_ONNX_MODEL_NAME, None, None)

Name                                                                                          Type                   Forward_MACs    FPercent    Memory      MPercent    Params     PPercent    InShape        OutShape
--------------------------------------------------------------------------------------------  ---------------------  --------------  ----------  ----------  ----------  ---------  ----------  -------------  -------------
Slice_0                                                                                       Slice                  0               0.00%       131,136     0.26%       8          0.00%       1x4x512x512    1x4x128x128
/eyebrow_decomposer/body/downsample_blocks.0/downsample_blocks.0.0/Conv                       Conv                   589,824         0.04%       131,144     0.26%       36         0.00%       1x4x128x128    1x4x128x128
/eyebrow_decomposer/body/downsample_blocks.0/downsample_blocks.0.1/Conv                       Conv                   4,194,30

In [10]:
# Small bench on cpu to check performance
for i in tqdm(range(100)):
    eyebrow_decomposer(eyebrow_decomposer_input_img)
for i in tqdm(range(100)):
    ort_sess.run(None, {'input_image':pt_img_np,})
#Since decomposer is not required for every run iteration, no need to quantize

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 150.19it/s]


In [11]:
# Start working on eyebrow morphine combiner

EYEBROW_MORPHING_COMBINER_INPUT_LIST = ['input_image','eyebrow_background_layer', "eyebrow_layer", 'eyebrow_pose']
EYEBROW_MORPHING_COMBINER_OUTPUT_LIST = ['eyebrow_image']  # 7
EYEBROW_POSE_SHAPE = (1, 12)
EYEBROW_MORPHING_COMBINER_ONNX_MODEL_NAME = "eyebrow_morphing_combiner.onnx"
EYEBROW_MORPHING_COMBINER_SIMPLIFIED_ONNX_MODEL_NAME = "eyebrow_morphing_combiner_sim.onnx"


eyebrow_pose_zero = torch.zeros(EYEBROW_POSE_SHAPE, dtype=dtype, device=device)
eyebrow_morphing_combiner_torch_res = eyebrow_morphing_combiner(eyebrow_decomposer_torch_res[3], 
                                                                eyebrow_decomposer_torch_res[0], eyebrow_pose_zero)

#Build a new eyebrow_morphing_combiner that does cropping
class EyebrowMorphingCombinerWrapper(Module):
    def __init__(self, eyebrow_morphing_combiner_obj):
        super().__init__()
        self.eyebrow_morphing_combiner = eyebrow_morphing_combiner_obj
    def forward(self, full_image:Tensor, background_layer: Tensor, eyebrow_layer: Tensor, pose: Tensor, *args) -> Tensor:
        im_morpher_crop = full_image[:, :, 32:32 + 192, (32 + 128):(32 + 192 + 128)].clone()
        im_morpher_crop[:, :, 32:32 + 128, 32:32 + 128] = self.eyebrow_morphing_combiner(background_layer, eyebrow_layer, pose)[2]
        return im_morpher_crop
eyebrow_morphing_combiner_wrapped = EyebrowMorphingCombinerWrapper(eyebrow_morphing_combiner).eval()
eyebrow_morphing_combiner_wrapped_torch_res = eyebrow_morphing_combiner_wrapped(pt_img, eyebrow_decomposer_wrapped_torch_res[0], 
                                                                                eyebrow_decomposer_wrapped_torch_res[1], eyebrow_pose_zero)


input_tuple = (pt_img, eyebrow_decomposer_wrapped_torch_res[0], eyebrow_decomposer_wrapped_torch_res[1], eyebrow_pose_zero)
input_dict = {k:v.cpu().detach().numpy() for k,v in zip(EYEBROW_MORPHING_COMBINER_INPUT_LIST,input_tuple)}
torch.onnx.export(eyebrow_morphing_combiner_wrapped,               # model being run
                  input_tuple,                         # model input (or a tuple for multiple inputs)
                  EYEBROW_MORPHING_COMBINER_ONNX_MODEL_NAME,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = EYEBROW_MORPHING_COMBINER_INPUT_LIST,   # the model's input names
                  output_names = EYEBROW_MORPHING_COMBINER_OUTPUT_LIST) 
onnx_model = onnx.load(EYEBROW_MORPHING_COMBINER_ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
eyebrow_morphing_combiner_onnx_model_sim, check = simplify(onnx_model)
if check:
    onnx.save(eyebrow_morphing_combiner_onnx_model_sim,EYEBROW_MORPHING_COMBINER_SIMPLIFIED_ONNX_MODEL_NAME)
else:
    print("Simplify error!")

im_morpher_crop = pt_img[:, :, 32:32 + 192, (32 + 128):(32 + 192 + 128)].clone()
im_morpher_crop[:, :, 32:32 + 128, 32:32 + 128] =eyebrow_morphing_combiner_torch_res[2]

# Verify correctness compare to pytorch
ort_sess_sim = ort.InferenceSession(EYEBROW_MORPHING_COMBINER_SIMPLIFIED_ONNX_MODEL_NAME, sess_options=sess_options, providers=providers)
onnx_sim_output = ort_sess_sim.run(None,input_dict)
print("MSE is: ",((onnx_sim_output[0] - im_morpher_crop.cpu().detach().numpy()) ** 2).mean())

F:\talking-head-anime-3-demo\tha3\nn\image_processing_util.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if n == self.last_n and device == self.last_device:


MSE is:  6e-08


In [12]:
def printInfo(a):
    print(a.dtype, a.shape, np.max(a),np.min(a), np.mean(a))
printInfo(onnx_sim_output[0])
printInfo(im_morpher_crop.cpu().detach().numpy())

float16 (1, 4, 192, 192) 1.0 -1.0 -0.684
float16 (1, 4, 192, 192) 1.0 -1.0 -0.684


In [13]:
onnx_tool.model_profile(EYEBROW_MORPHING_COMBINER_SIMPLIFIED_ONNX_MODEL_NAME, None, None)

Name                                                                                                     Type                   Forward_MACs    FPercent    Memory      MPercent    Params     PPercent    InShape        OutShape
-------------------------------------------------------------------------------------------------------  ---------------------  --------------  ----------  ----------  ----------  ---------  ----------  -------------  -------------
Slice_0                                                                                                  Slice                  0               0.00%       294,976     0.54%       8          0.00%       1x4x512x512    1x4x192x192
/eyebrow_morphing_combiner/Concat                                                                        Concat                 0               0.00%       262,144     0.48%       0          0.00%       1x4x128x128    1x8x128x128
/eyebrow_morphing_combiner/body/Reshape                                          

In [14]:
# Small bench on cpu to check performance
for i in tqdm(range(100)):
    eyebrow_morphing_combiner(eyebrow_decomposer_torch_res[3], eyebrow_decomposer_torch_res[0], eyebrow_pose_zero)
for i in tqdm(range(100)):
    ort_sess_sim.run(None,input_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 85.44it/s]


In [15]:
#Play with face morpher
FACE_POSE_SHAPE = (1,27)
face_pose_zero = torch.zeros(FACE_POSE_SHAPE, dtype=dtype, device=device)
face_morpher_torch_res = face_morpher(im_morpher_crop, face_pose_zero)
face_morphed_full = pt_img.clone()
face_morphed_full[:, :, 32:32 + 192, 32 + 128:32 + 192 + 128] = face_morpher_torch_res[0]
face_morphed_half = interpolate(face_morphed_full, size=(256, 256), mode='bilinear', align_corners=False)


class FaceMorpherWrapped(Module):
    def __init__(self, face_morpher_obj):
        super().__init__()
        self.face_morpher = face_morpher_obj
    def forward(self, input_image: Tensor, im_morpher_crop: Tensor, face_pose:Tensor,  *args) -> List[Tensor]:
        face_morphed_full = input_image.clone()
        face_morphed_full[:, :, 32:32 + 192, 32 + 128:32 + 192 + 128] = self.face_morpher(im_morpher_crop, face_pose)[0]
        face_morphed_half = interpolate(face_morphed_full, size=(256, 256), mode='bilinear', align_corners=False)
        return [face_morphed_full, face_morphed_half]
face_morpher_wrapped = FaceMorpherWrapped(face_morpher).eval()
face_morpher_wrapped_torch_res = face_morpher_wrapped(pt_img, eyebrow_morphing_combiner_wrapped_torch_res, face_pose_zero) #May be error

FACE_MORPHER_ONNX_MODEL_NAME = 'face_morpher.onnx'
FACE_MORPHER_SIM_ONNX_MODEL_NAME = 'face_morpher_sim.onnx'

FACE_MORPHER_OUTPUT_LIST = ['face_morphed_full', 'face_morphed_half']
FACE_MORPHER_INPUT_LIST = ['input_image', 'im_morpher_crop', 'face_pose']
input_tuple = (pt_img, eyebrow_morphing_combiner_wrapped_torch_res, face_pose_zero)
input_dict = {k:v.cpu().detach().numpy() for k,v in zip(FACE_MORPHER_INPUT_LIST,input_tuple)}

torch.onnx.export(face_morpher_wrapped,               # model being run
                  input_tuple,                         # model input (or a tuple for multiple inputs)
                  FACE_MORPHER_ONNX_MODEL_NAME,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = FACE_MORPHER_INPUT_LIST,   # the model's input names
                  output_names = FACE_MORPHER_OUTPUT_LIST) 

onnx_model = onnx.load(FACE_MORPHER_ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
onnx_model_sim, check = simplify(onnx_model)
if check:
    onnx.save(onnx_model_sim,FACE_MORPHER_SIM_ONNX_MODEL_NAME)
else:
    print("Simplify error!")

ort_sess_sim = ort.InferenceSession(FACE_MORPHER_SIM_ONNX_MODEL_NAME, sess_options=sess_options, providers=providers)
onnx_sim_output = ort_sess_sim.run(None,input_dict)

printInfo(onnx_sim_output[0])
printInfo(onnx_sim_output[1])
printInfo(face_morphed_full.cpu().detach().numpy())
printInfo(face_morphed_half.cpu().detach().numpy())
print("MSE is: ",((onnx_sim_output[0] - face_morphed_full.cpu().detach().numpy()) ** 2).mean())
print("MSE is: ",((onnx_sim_output[1] - face_morphed_half.cpu().detach().numpy()) ** 2).mean())

float16 (1, 4, 512, 512) 1.0 -1.0 -0.864
float16 (1, 4, 256, 256) 1.0 -1.0 -0.864
float16 (1, 4, 512, 512) 1.0 -1.0 -0.864
float16 (1, 4, 256, 256) 1.0 -1.0 -0.864
MSE is:  6.26e-06
MSE is:  2.1e-06


In [16]:
for i in tqdm(range(100)):
    face_morpher(im_morpher_crop, face_pose_zero)
for i in tqdm(range(100)):
    ort_sess_sim.run(None,input_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 75.14it/s]


In [17]:
# Try to split out the Encoder part of the mopher model
FACE_MORPHER_ENCODER = 'face_morpher_sim_encoder.onnx'
onnx.utils.extract_model(FACE_MORPHER_SIM_ONNX_MODEL_NAME, FACE_MORPHER_ENCODER, ['im_morpher_crop'], 
                         ['/face_morpher/body/downsample_blocks.3/downsample_blocks.3.3/Relu_output_0'])
onnx.checker.check_model(onnx.load(FACE_MORPHER_ENCODER))
FACE_MORPHER_NEW = 'face_morpher_new.onnx'
onnx.utils.extract_model(FACE_MORPHER_SIM_ONNX_MODEL_NAME, FACE_MORPHER_NEW, 
                         ['input_image','im_morpher_crop','face_pose',
                          '/face_morpher/body/downsample_blocks.3/downsample_blocks.3.3/Relu_output_0'], 
                         ['face_morphed_full', 'face_morphed_half'])
onnx.checker.check_model(onnx.load(FACE_MORPHER_NEW))

In [18]:
EYEBROW_COMBINER_NEW = 'eyebrow_morphing_combiner_new.onnx'
eyebrow_combiner_model =  onnx.load(EYEBROW_MORPHING_COMBINER_SIMPLIFIED_ONNX_MODEL_NAME)
face_morpher_encoder_model = onnx.load(FACE_MORPHER_ENCODER)

eyebrow_combiner_new_model = onnx.compose.merge_models(
    eyebrow_combiner_model, face_morpher_encoder_model,
    io_map=[("eyebrow_image", "im_morpher_crop")]
)
onnx.save(eyebrow_combiner_new_model, "temp.onnx")
onnx.utils.extract_model("temp.onnx", EYEBROW_COMBINER_NEW, ['input_image', 'eyebrow_background_layer', 'eyebrow_layer', 'eyebrow_pose'], 
                         ['eyebrow_image', '/face_morpher/body/downsample_blocks.3/downsample_blocks.3.3/Relu_output_0'])
onnx.checker.check_model(onnx.load(EYEBROW_COMBINER_NEW))

In [19]:
onnx_tool.model_profile("face_morpher_new.onnx", None, None)

Name                                                                                    Type                   Forward_MACs    FPercent    Memory      MPercent    Params     PPercent    InShape        OutShape
--------------------------------------------------------------------------------------  ---------------------  --------------  ----------  ----------  ----------  ---------  ----------  -------------  -------------
/face_morpher/body/Reshape                                                              Reshape                0               0.00%       86          0.00%       4          0.00%       1x27           1x27x1x1
/face_morpher/body/Tile                                                                 Tile                   0               0.00%       31,136      0.04%       4          0.00%       1x27x1x1       1x27x24x24
/face_morpher/body/Concat                                                               Concat                 0               0.00%       620,928     0.

In [20]:
ROTATION_POSE_SHAPE = (1,6)
rotation_pose_zero = torch.zeros(ROTATION_POSE_SHAPE, dtype=dtype, device=device)
two_algo_face_body_rotator_torch_res = two_algo_face_body_rotator(face_morphed_half, rotation_pose_zero)
class TwoAlgoFaceBodyRotatorWrapped(Module):
    def __init__(self, two_algo_face_body_rotator_obj):
        super().__init__()
        self.two_algo_face_body_rotator = two_algo_face_body_rotator_obj
    def forward(self, image: Tensor, pose: Tensor, *args) -> List[Tensor]:
        res = self.two_algo_face_body_rotator(image, pose)
        full_warped_image = interpolate(res[1], size=(512, 512), mode='bilinear', align_corners=False)
        full_grid_change = interpolate(res[2], size=(512, 512), mode='bilinear', align_corners=False)
        return [full_warped_image, full_grid_change]
two_algo_face_body_rotator_wrapped = TwoAlgoFaceBodyRotatorWrapped(two_algo_face_body_rotator)
rotator_wrapped_torch_res = two_algo_face_body_rotator_wrapped(face_morpher_wrapped_torch_res[1], rotation_pose_zero)

ROTATOR_ONNX_MODEL_NAME = 'two_algo_face_body_rotator.onnx'
ROTATOR_SIM_ONNX_MODEL_NAME = 'two_algo_face_body_rotator_sim.onnx'

ROTATOR_OUTPUT_LIST = ['full_warped_image', 'full_grid_change']
ROTATOR_INPUT_LIST = ['face_morphed_half', 'rotation_pose']
input_tuple = (face_morpher_wrapped_torch_res[1], rotation_pose_zero)
input_dict = {k:v.cpu().detach().numpy() for k,v in zip(ROTATOR_INPUT_LIST,input_tuple)}

torch.onnx.export(two_algo_face_body_rotator_wrapped,               # model being run
                  input_tuple,                         # model input (or a tuple for multiple inputs)
                  ROTATOR_ONNX_MODEL_NAME,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ROTATOR_INPUT_LIST,   # the model's input names
                  output_names = ROTATOR_OUTPUT_LIST) 

onnx_model = onnx.load(ROTATOR_ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
onnx_model_sim, check = simplify(onnx_model)
if check:
    onnx.save(onnx_model_sim,ROTATOR_SIM_ONNX_MODEL_NAME)
else:
    print("Simplify error!")

ort_sess_sim = ort.InferenceSession(ROTATOR_SIM_ONNX_MODEL_NAME, sess_options=sess_options, providers=providers)
onnx_sim_output = ort_sess_sim.run(None,input_dict)
full_warped_image = interpolate(two_algo_face_body_rotator_torch_res[1], size=(512, 512), mode='bilinear', align_corners=False)
full_grid_change = interpolate(two_algo_face_body_rotator_torch_res[2], size=(512, 512), mode='bilinear', align_corners=False)

printInfo(full_warped_image.cpu().detach().numpy())
printInfo(full_grid_change.cpu().detach().numpy())
printInfo(onnx_sim_output[0])
printInfo(onnx_sim_output[1])
print("MSE is: ",((onnx_sim_output[0] - full_warped_image.cpu().detach().numpy()) ** 2).mean())
print("MSE is: ",((onnx_sim_output[1] - full_grid_change.cpu().detach().numpy()) ** 2).mean())
onnx.checker.check_model(onnx.load(ROTATOR_SIM_ONNX_MODEL_NAME))

float16 (1, 4, 512, 512) 1.0 -1.0 -0.867
float16 (1, 2, 512, 512) 0.4192 -0.531 -0.005383
float16 (1, 4, 512, 512) 1.0 -1.0 -0.867
float16 (1, 2, 512, 512) 0.42 -0.531 -0.005383
MSE is:  2.3e-06
MSE is:  0.0


In [21]:
onnx_tool.model_profile(ROTATOR_SIM_ONNX_MODEL_NAME, None, None)

Name                                                                                                           Type                   Forward_MACs    FPercent    Memory       MPercent    Params     PPercent    InShape        OutShape
-------------------------------------------------------------------------------------------------------------  ---------------------  --------------  ----------  -----------  ----------  ---------  ----------  -------------  -------------
/two_algo_face_body_rotator/Reshape                                                                            Reshape                0               0.00%       44           0.00%       4          0.00%       1x6            1x6x1x1
/two_algo_face_body_rotator/Tile                                                                               Tile                   0               0.00%       786,464      0.36%       4          0.00%       1x6x1x1        1x6x256x256
/two_algo_face_body_rotator/Concat                       

In [22]:
editor_torch_res = editor(face_morphed_full, 
                          full_warped_image, 
                          full_grid_change, 
                          rotation_pose_zero)
class EditorWrapped(Module):
    def __init__(self, editor_obj):
        super().__init__()
        self.editor = editor_obj
    def forward(self,
                morphed_image: Tensor,
                rotated_warped_image: Tensor,
                rotated_grid_change: Tensor,
                pose: Tensor,
                *args) -> List[Tensor]:
        res = self.editor(morphed_image, rotated_warped_image, rotated_grid_change, pose)[0]
        return res.reshape(4, 512 * 512).transpose(0,1).reshape(512,512,4)
editor_wrapped = EditorWrapped(editor)
editor_wrapped_torch_res = editor_wrapped(face_morpher_wrapped_torch_res[0], 
                                          rotator_wrapped_torch_res[0], 
                                          rotator_wrapped_torch_res[1], 
                                          rotation_pose_zero)

EDITOR_ONNX_MODEL_NAME = 'editor.onnx'
EDITOR_SIM_ONNX_MODEL_NAME = 'editor_sim.onnx'

EDITOR_OUTPUT_LIST = ['result']
EDITOR_INPUT_LIST = ['morphed_image', 'rotated_warped_image','rotated_grid_change','rotation_pose']
input_tuple = (face_morpher_wrapped_torch_res[0], 
                          rotator_wrapped_torch_res[0], 
                          rotator_wrapped_torch_res[1], 
                          rotation_pose_zero)
input_dict = {k:v.cpu().detach().numpy() for k,v in zip(EDITOR_INPUT_LIST,input_tuple)}

torch.onnx.export(editor_wrapped,               # model being run
                  input_tuple,                         # model input (or a tuple for multiple inputs)
                  EDITOR_ONNX_MODEL_NAME,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = EDITOR_INPUT_LIST,   # the model's input names
                  output_names = EDITOR_OUTPUT_LIST) 

onnx_model = onnx.load(EDITOR_ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
onnx_model_sim, check = simplify(onnx_model)
if check:
    onnx.save(onnx_model_sim,EDITOR_SIM_ONNX_MODEL_NAME)
else:
    print("Simplify error!")

ort_sess_sim = ort.InferenceSession(EDITOR_SIM_ONNX_MODEL_NAME, sess_options=sess_options, providers=providers)
onnx_sim_output = ort_sess_sim.run(None,input_dict)

In [23]:
printInfo(onnx_sim_output[0])
poser_res = editor_torch_res[0].reshape(4, 512 * 512).transpose(0,1).reshape(512,512,4).cpu().detach().numpy()
printInfo(poser_res)
print("MSE is: ",((onnx_sim_output[0] - poser_res) ** 2).mean())

float16 (512, 512, 4) 1.0 -1.0 -0.868
float16 (512, 512, 4) 1.0 -1.0 -0.868
MSE is:  9.7e-06


In [24]:
from PIL import Image
def saveImg(path:str, arry):
    resImg = ((arry/2.0 + 0.5)*255).astype('uint8')
    Image.fromarray(resImg).convert('RGB').save(path)

In [25]:
saveImg('ref.png', poser_res)
saveImg('test.png', onnx_sim_output[0])

In [26]:
# So far model sequanse is :
[
    "eyebrow_decomposer_sim.onnx",
    'eyebrow_morphing_combiner_new.onnx',
    'face_morpher_new.onnx',
    'two_algo_face_body_rotator_sim.onnx',
    'editor_sim.onnx'
]

['eyebrow_decomposer_sim.onnx',
 'eyebrow_morphing_combiner_new.onnx',
 'face_morpher_new.onnx',
 'two_algo_face_body_rotator_sim.onnx',
 'editor_sim.onnx']

In [27]:
class RunTest():
    def __init__(self, img = None, ref = None):
        if HALF:
            self.dtype = np.float16
        else:
            self.dtype = np.float32
        self.decomposer_sess = ort.InferenceSession("eyebrow_decomposer_sim.onnx", sess_options=sess_options, providers=providers)
        self.combiner_sess = ort.InferenceSession("eyebrow_morphing_combiner_new.onnx", sess_options=sess_options, providers=providers)
        self.morpher_sess = ort.InferenceSession("face_morpher_new.onnx", sess_options=sess_options, providers=providers)
        self.rotator_sess = ort.InferenceSession("two_algo_face_body_rotator_sim.onnx", sess_options=sess_options, providers=providers)
        self.editor_sess = ort.InferenceSession("editor_sim.onnx", sess_options=sess_options, providers=providers)
        if img == None:
            self.img = np.random.rand(1, 4, 512, 512).astype(self.dtype) * 2.0 - 1.0
        else:
            self.img = img.cpu().detach().numpy()
        self.eyebrow_pose_zero = np.zeros((1,12), dtype=self.dtype)
        self.face_pose_zero = np.zeros((1,27), dtype=self.dtype)
        self.rotation_pose_zero = np.zeros((1,6), dtype=self.dtype)

        decomposer_res = self.decomposer_sess.run(None, {'input_image':self.img,})
        combiner_res = self.combiner_sess.run(None, {'input_image':self.img,
                                                     'eyebrow_background_layer': decomposer_res[0],
                                                     "eyebrow_layer": decomposer_res[1],
                                                     'eyebrow_pose':self.eyebrow_pose_zero,})
        morpher_res = self.morpher_sess.run(None, {'input_image':self.img,
                                                   'im_morpher_crop': combiner_res[0], 
                                                   'face_pose': self.face_pose_zero,
                                                   '/face_morpher/body/downsample_blocks.3/downsample_blocks.3.3/Relu_output_0':combiner_res[1]})
        rotator_res = self.rotator_sess.run(None, {'face_morphed_half':morpher_res[1], 
                                                   'rotation_pose':self.rotation_pose_zero})
        editor_res = self.editor_sess.run(None, {'morphed_image':morpher_res[0],
                                                 'rotated_warped_image':rotator_res[0],
                                                 'rotated_grid_change': rotator_res[1], 
                                                 'rotation_pose':self.rotation_pose_zero})
        if ref != None:
            def printInfo(a):
                print(a.dtype, a.shape, np.max(a),np.min(a), np.mean(a), np.sum(a))
            ref_np = ref.reshape(4, 512 * 512).transpose(0,1).reshape(512,512,4).cpu().detach().numpy()
            printInfo(editor_res[0])
            printInfo(ref_np)
            print("MSE is: ",((editor_res[0] - ref_np) ** 2).mean())
            from PIL import Image
            def saveImg(path:str, arry):
                resImg = ((arry/2.0 + 0.5)*255).astype('uint8')
                Image.fromarray(resImg).convert('RGB').save(path)
            saveImg('test_res.jpg',editor_res[0])
            saveImg('ref.jpg', ref_np)
RunTest(pt_img, poser_torch_res[0])

F:\Anaconda\envs\talking-head-anime-3-dev\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


float16 (512, 512, 4) 1.0 -1.0 -0.868 -inf
float16 (512, 512, 4) 1.0 -1.0 -0.868 -inf
MSE is:  1.85e-05
